In [50]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load_and_eval_atck_model, create_eval_post_loader, create_shadow_post_train_loader, evaluate_attack_model, DatasetClassN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Parameters

In [51]:
# Model Parameter
model = "mobilenetv2"
model_short = "mobilenet"
MODEL_MODULE = mobilenet_v2
# Dataset parameter
dataset = "cifar10"
dataset_short = "cifar"
DATASET_ENUM = DatasetClassN.Cifar

## Attack model training parameters
LEARNING_R = 0.01
EPOCHS = 5
MULTI_WORKERS_N = 4
STANDARDIZE = True

## Datasets
SHADOW_DATA_PATH = f"pickle/{dataset}/{model}/shadow.p"
EVALUATE_DATA_PATH = f"pickle/{dataset}/{model}/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = f"pickle/{dataset}/{model}/attack_train.p"
## Models
SHADOW_MODEL_PATH = f"shadow_models/{model}_shadow_{dataset_short}_overtrained.pth"
TARGET_MODEL_PATH = f"models/{model}_{dataset}.pth"
ATTACK_MODEL_PATH = f"attack_models/attack_{model_short}_{dataset_short}.pth"

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [52]:
#load the shadow model trained in the other python script
shadow_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value).to(DEVICE) #resnet_target is the shadow model
shadow_model.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))
pass

Shadow dataset

In [53]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
shadow_memb_data, shadow_non_memb_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
shadow_membloader = DataLoader(shadow_memb_data, batch_size=1, shuffle=False, num_workers=1)
shadow_non_membloader =  DataLoader(shadow_non_memb_data, batch_size=1, shuffle=True, num_workers=1)


### Attack Training Dataset

Create Attack model training dataset

In [54]:
attack_train_loader = create_shadow_post_train_loader(shadow_non_membloader, shadow_membloader, shadow_model, batch_size=64, \
    multi_n= MULTI_WORKERS_N, data_class = DATASET_ENUM, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH, standardize=STANDARDIZE)

NON Members
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 15000
Batchsize: 1
Inputs:tensor([[[[0.6275, 0.6431, 0.5922,  ..., 0.6471, 0.6392, 0.6510], [0.6588, 0.6784, 0.6275,  ..., 0.6980, 0.6667, 0.6784], [0.6667, 0.6863, 0.6706,  ..., 0.7020, 0.6667, 0.6745], ..., [0.3137, 0.2941, 0.2902,  ..., 0.5216, 0.4471, 0.5686], [0.3529, 0.3137, 0.2471,  ..., 0.4863, 0.5961, 0.7451], [0.3529, 0.3137, 0.2275,  ..., 0.5529, 0.7725, 0.7686]],[[0.5294, 0.5412, 0.4784,  ..., 0.5569, 0.5490, 0.5529], [0.5569, 0.5686, 0.5098,  ..., 0.6196, 0.5843, 0.5961], [0.5529, 0.5686, 0.5451,  ..., 0.6235, 0.5922, 0.5961], ..., [0.2314, 0.2235, 0.2235,  ..., 0.3922, 0.3333, 0.4549], [0.2510, 0.2275, 0.1725,  ..., 0.3529, 0.4824, 0.6314], [0.2431, 0.2196, 0.1529,  ..., 0.4314, 0.6706, 0.6627]],[[0.4039, 0.4078, 0.3412,  ..., 0.3137, 0.3137, 0.3412], [0.4314, 0.4353, 0.3686,  ..., 0.3647, 0.3294, 0.3412], [0.4314, 0.4392, 0.4078,  ..., 0.3725, 0.3333, 0.3333], ..., [0

In [ ]:
attack_sample_iter = iter(attack_train_loader)
print (str(next(attack_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 0.3991,  0.2773,  0.2371,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.3922,  0.2946,  0.1755,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.4578,  0.1725,  0.1703,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.1503,  0.1348,  0.0235,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.4818,  0.4373,  0.1972,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.8340,  0.4606,  0.4531,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.6342,  0.5083,  0.2848,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.6586,  0.5266,  0.3580,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000]],  [[ 0.7082,  0.5

### Attack Model

Simple Model

In [ ]:
class SmallAttackNN(nn.Module):
    def __init__(self):
        super(SmallAttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.fc2 = nn.Linear(32, 1)
       

    def forward(self, x):
        
        x = torch.sigmoid(self.fc2(self.fc1(x)))
        return x

Middle Size Model

In [ ]:
class MiddleAttackNN(nn.Module):
    def __init__(self):
        super(MiddleAttackNN, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x

Load Target Model

In [ ]:
# Load target model
target_model = MODEL_MODULE(weights=None,num_classes=DATASET_ENUM.value)
target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
target_model.eval()
target_model.to(DEVICE)
pass

### Evaluation Dataset

Load **Evaluation Dataset** & get Posteriors/ Member Labels 

In [ ]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    # Create Posteriors with target model; MULTI_WORKERS_N defines workers num of returned DL
    attack_eval_post_loader = create_eval_post_loader (target_model, eval_dataset, MULTI_WORKERS_N, DATASET_ENUM, DEVICE, test_dataset=False, standardize=STANDARDIZE)

------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 200
Batchsize: 1
Inputs:tensor([[[[0.8941, 0.9216, 0.9294,  ..., 0.1765, 0.4000, 0.7059], [0.9451, 0.9569, 0.9765,  ..., 0.1882, 0.6667, 0.6157], [0.8941, 0.9569, 0.9843,  ..., 0.1373, 0.3961, 0.3961], ..., [0.2784, 0.2510, 0.2392,  ..., 0.3412, 0.3137, 0.2941], [0.3961, 0.3333, 0.2392,  ..., 0.3137, 0.2980, 0.2745], [0.4275, 0.3647, 0.2353,  ..., 0.3137, 0.3020, 0.3020]],[[0.9882, 0.9843, 0.9882,  ..., 0.2667, 0.5176, 0.8078], [1.0000, 0.9961, 1.0000,  ..., 0.2667, 0.7647, 0.7059], [0.9216, 0.9804, 0.9922,  ..., 0.1961, 0.5020, 0.5216], ..., [0.3059, 0.3137, 0.3216,  ..., 0.4078, 0.3725, 0.3176], [0.4118, 0.3882, 0.3098,  ..., 0.4039, 0.3686, 0.3176], [0.4471, 0.3882, 0.2745,  ..., 0.3608, 0.3451, 0.3451]],[[0.9882, 0.9843, 0.9882,  ..., 0.3137, 0.6392, 0.8196], [1.0000, 0.9961, 1.0000,  ..., 0.2863, 0.7843, 0.6549], [0.9294, 0.9843, 0.9922,  ..., 0.2353, 0.5137, 0.4863], ..., [0.2353, 0.2510,

Attack Evaluation DL Sample

In [ ]:
eval_sample_iter = iter(attack_eval_post_loader)
print (str(next(eval_sample_iter)).replace('\n',"").replace("   ",""))

[tensor([[[ 1.0237e+01,  5.1730e-01, -3.0826e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 9.4312e+00,  5.0981e-01, -2.7521e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 1.1062e+01,  6.9038e-01, -4.8954e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 8.8281e+00,  7.7090e-01,  1.4212e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 9.5954e+00,  8.7125e-01,  2.9888e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],  [[ 8.4472e+00,  1.1986e-01,  1.0038e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0

### Train Attack model

In [ ]:
attack_model = MiddleAttackNN()
attack_model.to(DEVICE)
# Further training Parameters 
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)
# Do everything in one function
# Train/ Load Attack model; For every epoch show epoch loss and evaluate
train_or_load_and_eval_atck_model(attack_model, attack_train_loader, attack_eval_post_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH, DEVICE)


Epoch: 1
------SAMPLE WINDOW---------------------------------------------------------
Number Samples: 30000
Batchsize: 64
Inputs:tensor([[[ 1.1481,  0.6520,  0.6330,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.1025,  0.0685,  0.0503,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.5624,  0.2918,  0.1066,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.3418,  0.2416,  0.2339,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.6157,  0.3119,  0.1775,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.3105,  0.2277,  0.2219,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],  [[ 0.6464,  0.2648,  0.1184,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000]],  [[ 0.5389

KeyboardInterrupt: 

### Evaluation

In [ ]:
evaluate_attack_model(attack_model, attack_eval_post_loader, DEVICE)

Accuracy: 0.47


0.475